In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/students-performance-in-exams/StudentsPerformance.csv')
df.head()

## Pre-Processing

In [ ]:
df['percentage'] = round((df['math score'] + df['reading score'] + df['writing score']) / 3, 2)

In [ ]:
ranges = [0, 35, 50, 75, np.inf]
label = ['Fail', '3rd division', '2nd division', '1st division']

df['division'] = pd.cut(df['percentage'], bins=ranges, labels=label)

## Basic EDA

In [ ]:
sns.displot(df['percentage'], kde=True)
display(np.std(df['percentage']))

In [ ]:
sns.displot(df[df['gender'] == 'male']['percentage'], kde=True)
sns.displot(df[df['gender'] == 'female']['percentage'], kde=True)

plt.show()

In [ ]:
sns.heatmap(df.corr())
plt.show()

## Q. Does gender affect marks?

In [ ]:
sns.catplot(x='gender', kind='count', hue='division', data=df)
plt.show()

#### Ans: Probablity of females getting 1st or 2nd division is high

## Q. Does any group perform better than other groups?

In [ ]:
sns.countplot(x='race/ethnicity', data=df)
plt.show()

In [ ]:
sns.catplot(x='race/ethnicity', y='percentage', kind='bar', data=df)
plt.show()

In [ ]:
sns.catplot(x='gender', y='percentage', hue='race/ethnicity', kind='box', data=df)
plt.show()

#### Ans: 'group E' performed better than other groups and females are still ahead

## Q. Does parental degree affect the scores of there child?

In [ ]:
odr = ['some high school', 'high school', "associate's degree", 'some college', "bachelor's degree", "master's degree"]

sns.catplot(x='parental level of education', y='percentage', kind='swarm', order=odr, ci=None, data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.catplot(x='parental level of education', y='percentage', kind='point', ci=None, order=odr, data=df)
plt.xticks(rotation=90)
plt.show()

#### Ans: Yes, parents education does affect percentage of there child. 

## Q. Does lunch time affect score?

In [ ]:
sns.catplot(x='lunch', hue='division', kind='count', data=df)

#### Ans: Yes, it does affect. We can clearly see students without standard lunch time have high percentage of failure.

## Q. Does test preparation affects score?

In [ ]:
sns.countplot(df['test preparation course'])
plt.show()

In [ ]:
sns.catplot(x='test preparation course', hue='division', kind='count', data=df)
plt.show()

#### Ans: Yes, it does affect. Although those who didn't attended course have high number of 2nd division students, but the probability of having 1st or 2nd division is high for students who attended course and there is only one student who failed.

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Instantiate LabelEncoder
le=LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in df.columns:
    # Compare if the dtype is object
    if df[col].dtypes=='object':
    # Use LabelEncoder to do the numeric transformation
        df[col]=le.fit_transform(df[col])
       
X = df.drop('division', axis=1)
y = df.division
        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
best = {}
for i in range(1, 100):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_res, y_train_res)
    y_pred = knn.predict(X_test)
    acc = knn.score(X_test, y_test)
    #print(i,':',acc)
    best[i] = round(acc, 3)
    
plt.plot(best.keys(), best.values())
plt.xticks([i for i in range(0, 100, 5)])
plt.grid(True)
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
para = {'n_neighbors':np.arange(1, 50)}

grid_cv = GridSearchCV(knn, param_grid=para, cv=5)

grid_cv.fit(X_train_res, y_train_res)
grid_cv.best_params_

In [ ]:
knn = grid_cv.best_estimator_

knn.fit(X_train_res, y_train_res)

y_pred = knn.predict(X_test)

In [ ]:
knn.score(X_test, y_test)

In [ ]:
display(X_test.iloc[0]['percentage'])
display(knn.predict([X_test.iloc[0].tolist()]))

In [ ]:
original = y_test
data = {'original':original.to_list(),
        'prediction':y_pred.tolist()}
pd.DataFrame(data).head()